# IMDB 영화리뷰 감성분석 - Conv1D
- Conv1D + Conv1D + Dense
- Conv1D + LSTM + Dense
- Conv1D + Dense + Dense

In [1]:
import numpy as np
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

In [2]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.layers import Conv1D, MaxPooling1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
num_words = 10000
(X_train, y_train), (_,_) = imdb.load_data(num_words=num_words)

17473536/17464789 [==============================] - 0s 0us/step


In [4]:
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train, stratify=y_train, test_size=0.2, random_state=seed
)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((20000,), (5000,), (20000,), (5000,))

In [5]:
max_len = 500
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)
X_train.shape, X_test.shape

((20000, 500), (5000, 500))

- Case 1) Conv1D x 2

In [6]:
model1 = Sequential([
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 7, activation='relu'),
    MaxPooling1D(7),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(1, activation='sigmoid')
])
model1.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 100)          1000000   
                                                                 
 dropout (Dropout)           (None, 500, 100)          0         
                                                                 
 conv1d (Conv1D)             (None, 494, 64)           44864     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 70, 64)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 66, 64)            20544     
                                                                 
 max_pooling1d_1 (MaxPooling  (None, 13, 64)           0         
 1D)                                                    

In [7]:
model1.compile('adam', 'binary_crossentropy', ['accuracy'])
model1_path = 'best-imdb-conv1d-conv1d.h5'
mc1 = ModelCheckpoint(model1_path, verbose=1, save_best_only=True)
es1 = EarlyStopping(patience=5)

In [8]:
hist1 = model1.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc1, es1]
)

Epoch 1/30
249/250 [============================>.] - ETA: 0s - loss: 0.5573 - accuracy: 0.6782
Epoch 1: val_loss improved from inf to 0.36043, saving model to best-imdb-conv1d-conv1d.h5
250/250 [==============================] - 15s 10ms/step - loss: 0.5566 - accuracy: 0.6787 - val_loss: 0.3604 - val_accuracy: 0.8470
Epoch 2/30
248/250 [============================>.] - ETA: 0s - loss: 0.2876 - accuracy: 0.8838
Epoch 2: val_loss improved from 0.36043 to 0.32232, saving model to best-imdb-conv1d-conv1d.h5
250/250 [==============================] - 2s 9ms/step - loss: 0.2882 - accuracy: 0.8834 - val_loss: 0.3223 - val_accuracy: 0.8630
Epoch 3/30
250/250 [==============================] - ETA: 0s - loss: 0.1949 - accuracy: 0.9262
Epoch 3: val_loss did not improve from 0.32232
250/250 [==============================] - 2s 9ms/step - loss: 0.1949 - accuracy: 0.9262 - val_loss: 0.3523 - val_accuracy: 0.8555
Epoch 4/30
245/250 [============================>.] - ETA: 0s - loss: 0.1322 - accur

In [9]:
best_model1 = load_model(model1_path)
best_model1.evaluate(X_test, y_test)

157/157 [==============================] - 1s 3ms/step - loss: 0.3078 - accuracy: 0.8664


[0.30781808495521545, 0.8664000034332275]

- Case 2) Conv1D + LSTM

In [10]:
model2 = Sequential([
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    LSTM(100),
    Dense(1, activation='sigmoid')
])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_1 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_2 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 lstm (LSTM)                 (None, 100)               66000     
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                      

In [11]:
model2.compile('adam', 'binary_crossentropy', ['accuracy'])
model2_path = 'best-imdb-conv1d-lstm.h5'
mc2 = ModelCheckpoint(model2_path, verbose=1, save_best_only=True)
es2 = EarlyStopping(patience=5)

In [12]:
hist2 = model2.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc2, es2]
)

Epoch 1/30
246/250 [============================>.] - ETA: 0s - loss: 0.4435 - accuracy: 0.7654
Epoch 1: val_loss improved from inf to 0.29760, saving model to best-imdb-conv1d-lstm.h5
250/250 [==============================] - 6s 15ms/step - loss: 0.4409 - accuracy: 0.7676 - val_loss: 0.2976 - val_accuracy: 0.8802
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.2175 - accuracy: 0.9153
Epoch 2: val_loss improved from 0.29760 to 0.26507, saving model to best-imdb-conv1d-lstm.h5
250/250 [==============================] - 3s 13ms/step - loss: 0.2175 - accuracy: 0.9153 - val_loss: 0.2651 - val_accuracy: 0.8905
Epoch 3/30
247/250 [============================>.] - ETA: 0s - loss: 0.1418 - accuracy: 0.9488
Epoch 3: val_loss did not improve from 0.26507
250/250 [==============================] - 3s 13ms/step - loss: 0.1421 - accuracy: 0.9486 - val_loss: 0.2963 - val_accuracy: 0.8795
Epoch 4/30
246/250 [============================>.] - ETA: 0s - loss: 0.0968 - accuracy

In [13]:
best_model2 = load_model(model2_path)
best_model2.evaluate(X_test, y_test)

157/157 [==============================] - 1s 6ms/step - loss: 0.2719 - accuracy: 0.8884


[0.2718685567378998, 0.8884000182151794]

- Case 3) Conv1D + Dense

In [14]:
model3 = Sequential([
    Embedding(num_words, 100, input_length=max_len),
    Dropout(0.5),
    Conv1D(64, 5, activation='relu'),
    MaxPooling1D(5),
    GlobalMaxPooling1D(),
    Dense(100, activation='relu'),
    Dense(1, activation = 'sigmoid')
])
model3.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 500, 100)          1000000   
                                                                 
 dropout_2 (Dropout)         (None, 500, 100)          0         
                                                                 
 conv1d_3 (Conv1D)           (None, 496, 64)           32064     
                                                                 
 max_pooling1d_3 (MaxPooling  (None, 99, 64)           0         
 1D)                                                             
                                                                 
 global_max_pooling1d_1 (Glo  (None, 64)               0         
 balMaxPooling1D)                                                
                                                                 
 dense_2 (Dense)             (None, 100)              

In [15]:
model3.compile('adam', 'binary_crossentropy', ['accuracy'])
model3_path = 'best-imdb-conv1d-dense.h5'
mc3 = ModelCheckpoint(model3_path, verbose=1, save_best_only=True)
es3 = EarlyStopping(patience=5)

In [16]:
hist3 = model3.fit(
    X_train, y_train, validation_split=0.2,
    epochs=30, batch_size=64, callbacks=[mc3, es3]
)

Epoch 1/30
246/250 [============================>.] - ETA: 0s - loss: 0.5822 - accuracy: 0.6627
Epoch 1: val_loss improved from inf to 0.38916, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 3s 9ms/step - loss: 0.5789 - accuracy: 0.6654 - val_loss: 0.3892 - val_accuracy: 0.8288
Epoch 2/30
250/250 [==============================] - ETA: 0s - loss: 0.3149 - accuracy: 0.8666
Epoch 2: val_loss improved from 0.38916 to 0.29991, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 2s 8ms/step - loss: 0.3149 - accuracy: 0.8666 - val_loss: 0.2999 - val_accuracy: 0.8773
Epoch 3/30
248/250 [============================>.] - ETA: 0s - loss: 0.2106 - accuracy: 0.9183
Epoch 3: val_loss improved from 0.29991 to 0.29449, saving model to best-imdb-conv1d-dense.h5
250/250 [==============================] - 2s 8ms/step - loss: 0.2101 - accuracy: 0.9186 - val_loss: 0.2945 - val_accuracy: 0.8790
Epoch 4/30
249/250 [=====================

In [17]:
best_model3 = load_model(model3_path)
best_model3.evaluate(X_test, y_test)

157/157 [==============================] - 1s 3ms/step - loss: 0.2939 - accuracy: 0.8802


[0.29385656118392944, 0.8802000284194946]